In [1]:
import sys
sys.path.insert(0, '/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Canada/')

In [2]:
import torch
import pandas as pd
import numpy as np
import statistics
import torch
import random
import time
import numpy as np
#from transformers import AdamW
from torch.utils.tensorboard import SummaryWriter
import pickle
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import shutil
import itertools
import os
from pathlib import Path

from model.GRU_model import *
from pipeline_config import *
from utils import *

In [3]:
# Seeds for comparisons:

torch.manual_seed(1)
np.random.seed(2)
random.seed(3)
torch.use_deterministic_algorithms(True)


In [4]:
with open(train_dataset_path, 'rb') as f:
    train_dataset_dict = pickle.load(f)
    
with open(test_dataset_path, 'rb') as f:
    test_dataset_dict = pickle.load(f)

with open(category_id_to_category_name_path, 'rb') as f:
    category_id_to_name_dict = pickle.load(f)
    
with open(categories_per_indent_path, 'rb') as f:
    categories_per_indent_dict = pickle.load(f)

with open(son_parent_path, 'rb') as f:
    son_parent_dict = pickle.load(f)

with open(parent_to_son_list_path, 'rb') as f:
    parent_to_son_list_dict = pickle.load(f)

with open(hgru_model_weights_path, 'rb') as f:
    hgru_weight_dict = pickle.load(f)

with open(coefficient_dict_path, 'rb') as f:
    coefficient_dict = pickle.load(f)

In [5]:
print(f'Features: {Features}')
print(f'HiddenSize: {HiddenSize}')
print(f'LayersDim: {LayersDim}')
print(f'OutputDim: {OutputDim}')
print(f'DropoutProb: {DropoutProb}')
print(f'Lr : {Lr}')
print(f'Epochs: {Epochs}')

Features: 1
HiddenSize: 64
LayersDim: 1
OutputDim: 1
DropoutProb: 0.0
Lr : 0.1
Epochs: 100


In [6]:
def get_results_on_test_set(weights_path, train_dataset_dict, test_dataset_dict, categories = None):
    predictions_dict = {}
    if categories is None:
        categories = list(test_dataset_dict.keys())
        
    for category in categories:
        if category == 'Food and beverages':
            continue
        print(category)
        train_dataloader, test_dataloader = create_dataloader(train_dataset_dict[category], test_dataset_dict[category])
        basic_model =GRUModel(input_dim=Features, hidden_dim=HiddenSize, layer_dim=LayersDim, output_dim=OutputDim, dropout_prob=DropoutProb, seed=0)
        basic_optimizer = torch.optim.AdamW(basic_model.parameters(), lr=Lr)
        ckp_path = weights_path+category+'.pt'
        model, optimizer, checkpoint, valid_loss_min = load_checkpoint(ckp_path, basic_model, basic_optimizer)
        predictions = get_predictions_on_test_set(model, test_dataloader)
        predictions_dict[category] = predictions
    return predictions_dict

In [7]:
def rmdir(dirc):
    dirc = Path(dirc)
    for itm in dirc.iterdir():
        if itm.is_dir():
            rmdir(itm)
            print("Deleting", itm, ".... successful.")
        else:
            itm.unlink()
    dirc.rmdir()

# Loss Analysis 1:

No Loss Term 1:

In [8]:
# Sanity Check:
weightspath_1

'/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Canada/bidirectional/models_weights_1/'

In [9]:
path = os.getcwd()
desired_path = path + '/models_weights_1/'
desired_path

'/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Canada/bidirectional/models_weights_1/'

In [10]:
def bi_directional_model_1(son_parent_dict, parent_to_son_list_dict, train_dataset_dict, test_dataset_dict, categories_per_indent_dict, category_id_to_name_dict, weights_path, hgru_weight_dict, coefficient_dict,loss_coef_1, loss_coef_2, loss_coef_3, alpha):
    # deleting files in models_weights folder
    path = os.getcwd()
    desired_path = path + '/models_weights_1/'
    is_empty = not any(Path(desired_path).iterdir())

    if is_empty==False:
        rmdir(Path(desired_path))
    
    # deleting empty folder
    try:
        os.rmdir(desired_path)
    except OSError:
        print ("Deletion of the directory %s failed" % desired_path)
    else:
        print ("Successfully deleted the directory %s" % desired_path)

    # creating new folder
    try:
        os.mkdir(desired_path)
    except OSError:
        print ("Creation of the directory %s failed" % desired_path)
    else:
        print ("Successfully created the directory %s" % desired_path)
    #-----------------------------------------------------------------------------------------------------------------------------------------------#

    num_categories = 0
    loss_coef_1= 0
    loss_coef_2= 1*np.exp(-10)
    loss_coef_3= 6.16933911191657*np.exp(-8)
    alpha= 1.0

    for indent in sorted(list(categories_per_indent_dict.keys()),reverse=True):
        for category in categories_per_indent_dict[indent]:
            num_categories +=1
            print(f'num categories: {num_categories}')
            category_name = category_id_to_name_dict[category]
            print(f'category id|name: {category}|{category_name}')

            if int(indent) == 0 or son_parent_dict[category] not in categories_per_indent_dict[indent-1]: #or (category, son_parent_dict[category]) not in son_parent_corr_dict.keys():
                loss_coef_1 = 0

            if (category not in list(parent_to_son_list_dict.keys())) or (set(parent_to_son_list_dict[category]['sons']).issubset(set(categories_per_indent_dict[indent+1]))):
                loss_coef_2 = 0
            
            print('------------------------------------------------------------------')

            train_dataloader, test_dataloader = create_dataloader(train_dataset_dict[category_name], test_dataset_dict[category_name])
            model = GRUModel(input_dim=Features, hidden_dim=HiddenSize, layer_dim=LayersDim, output_dim=OutputDim, dropout_prob=DropoutProb, seed=0)
            optimizer = torch.optim.AdamW(model.parameters(), lr=Lr)
            model.to(Device)
            saving_param_path = weights_path+category_name+'.pt'
            
            training_and_evaluation(model, indent, train_dataloader, test_dataloader, optimizer, category, hgru_weight_dict, coefficient_dict, son_parent_dict, parent_to_son_list_dict, category_id_to_name_dict, loss_coef_1, loss_coef_2, loss_coef_3, alpha, saving_param_path)

In [11]:
bidrectional_models_1 = bi_directional_model_1(son_parent_dict, parent_to_son_list_dict, train_dataset_dict, test_dataset_dict, categories_per_indent_dict, category_id_to_name_dict, weightspath_1, hgru_weight_dict, coefficient_dict, 0, loss_coef_2, loss_coef_3, alpha)

Deletion of the directory /Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Canada/bidirectional/models_weights_1/ failed
Successfully created the directory /Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Canada/bidirectional/models_weights_1/
num categories: 1
category id|name: 345|All other miscellaneous food preparations
------------------------------------------------------------------
num categories: 2
category id|name: 316|Bacon
------------------------------------------------------------------
num categories: 3
category id|name: 332|Canned fruit
------------------------------------------------------------------
num categories: 4
category id|name: 341|Canned infant or junior foods
------------------------------------------------------------------
num categories: 5
category id|name: 318|Canned salmon
------------------------------------------------------------------
num categories: 6
category id|name: 319|Canned tuna
----------------------------------

Save prediction dict:

In [13]:
categories_lists = list(categories_per_indent_dict.values())
categories_id = list(itertools.chain.from_iterable(categories_lists))
categories = []
for category_id in categories_id:
    categories.append(category_id_to_name_dict[category_id])

In [19]:
predictions_dict_1 = get_results_on_test_set(weightspath_1, train_dataset_dict, test_dataset_dict, categories = categories)
with open(test_predictions_path_1, 'wb') as handle:
    pickle.dump(predictions_dict_1, handle, protocol=pickle.HIGHEST_PROTOCOL)

Air transportation
Audio equipment
Bakery products
Beer purchased from stores
Beer served in licensed establishments
Books and reading material (excluding textbooks
Cereal products (excluding baby food
Child care services
City bus and subway transportation
Coffee and tea
Condiments, spices and vinegars
Dairy products
Dental care services
Detergents and soaps (other than personal care
Digital computing equipment and devices
Edible fats and oils
Eggs
Eye care goods
Eye care services
Financial services
Fish
Fresh fruit
Fresh or frozen meat (excluding poultry
Fresh or frozen poultry
Fresh vegetables
Furniture
Gasoline
Household appliances
Household textiles
Housekeeping services
Internet access services
Liquor purchased from stores
Liquor served in licensed establishments
Magazines and periodicals
Medicinal and pharmaceutical products
Newspapers
Non-alcoholic beverages
Non-electric kitchen utensils, tableware and cookware
Nuts and seeds
Operation of recreational vehicles
Oral-hygiene produ

# Loss Analysis 2:

No Loss Term 2:

In [14]:
# Sanity Check:
weightspath_2

'/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Canada/bidirectional/models_weights_2/'

In [15]:
def bi_directional_model_2(son_parent_dict, parent_to_son_list_dict, train_dataset_dict, test_dataset_dict, categories_per_indent_dict, category_id_to_name_dict, weights_path, hgru_weight_dict, coefficient_dict,loss_coef_1, loss_coef_2, loss_coef_3, alpha):
    # deleting files in models_weights folder
    path = os.getcwd()
    desired_path = path + '/models_weights_2/'
    is_empty = not any(Path(desired_path).iterdir())

    if is_empty==False:
        rmdir(Path(desired_path))
    
    # deleting empty folder
    try:
        os.rmdir(desired_path)
    except OSError:
        print ("Deletion of the directory %s failed" % desired_path)
    else:
        print ("Successfully deleted the directory %s" % desired_path)

    # creating new folder
    try:
        os.mkdir(desired_path)
    except OSError:
        print ("Creation of the directory %s failed" % desired_path)
    else:
        print ("Successfully created the directory %s" % desired_path)
    #-----------------------------------------------------------------------------------------------------------------------------------------------#

    num_categories = 0
    loss_coef_1= 1*np.exp(-10)
    loss_coef_2= 0
    loss_coef_3= 6.16933911191657*np.exp(-8)
    alpha= 1.0

    for indent in sorted(list(categories_per_indent_dict.keys()),reverse=True):
        for category in categories_per_indent_dict[indent]:
            num_categories +=1
            print(f'num categories: {num_categories}')
            category_name = category_id_to_name_dict[category]
            print(f'category id|name: {category}|{category_name}')

            if int(indent) == 0 or son_parent_dict[category] not in categories_per_indent_dict[indent-1]: #or (category, son_parent_dict[category]) not in son_parent_corr_dict.keys():
                loss_coef_1 = 0

            if (category not in list(parent_to_son_list_dict.keys())) or (set(parent_to_son_list_dict[category]['sons']).issubset(set(categories_per_indent_dict[indent+1]))):
                loss_coef_2 = 0
            
            print('------------------------------------------------------------------')

            train_dataloader, test_dataloader = create_dataloader(train_dataset_dict[category_name], test_dataset_dict[category_name])
            model = GRUModel(input_dim=Features, hidden_dim=HiddenSize, layer_dim=LayersDim, output_dim=OutputDim, dropout_prob=DropoutProb, seed=0)
            optimizer = torch.optim.AdamW(model.parameters(), lr=Lr)
            model.to(Device)
            saving_param_path = weights_path+category_name+'.pt'
            
            training_and_evaluation(model, indent, train_dataloader, test_dataloader, optimizer, category, hgru_weight_dict, coefficient_dict, son_parent_dict, parent_to_son_list_dict, category_id_to_name_dict, loss_coef_1, loss_coef_2, loss_coef_3, alpha, saving_param_path)

In [16]:
bidrectional_models_2 = bi_directional_model_2(son_parent_dict, parent_to_son_list_dict, train_dataset_dict, test_dataset_dict, categories_per_indent_dict, category_id_to_name_dict, weightspath_2, hgru_weight_dict, coefficient_dict, loss_coef_1, 0, loss_coef_3, alpha)

Deletion of the directory /Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Canada/bidirectional/models_weights_2/ failed
Successfully created the directory /Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Canada/bidirectional/models_weights_2/
num categories: 1
category id|name: 345|All other miscellaneous food preparations
------------------------------------------------------------------
num categories: 2
category id|name: 316|Bacon
------------------------------------------------------------------
num categories: 3
category id|name: 332|Canned fruit
------------------------------------------------------------------
num categories: 4
category id|name: 341|Canned infant or junior foods
------------------------------------------------------------------
num categories: 5
category id|name: 318|Canned salmon
------------------------------------------------------------------
num categories: 6
category id|name: 319|Canned tuna
----------------------------------

In [17]:
predictions_dict_2 = get_results_on_test_set(weightspath_2, train_dataset_dict, test_dataset_dict, categories = categories)
with open(test_predictions_path_2, 'wb') as handle:
    pickle.dump(predictions_dict_2, handle, protocol=pickle.HIGHEST_PROTOCOL)

Air transportation
Audio equipment
Bakery products
Beer purchased from stores
Beer served in licensed establishments
Books and reading material (excluding textbooks
Cereal products (excluding baby food
Child care services
City bus and subway transportation
Coffee and tea
Condiments, spices and vinegars
Dairy products
Dental care services
Detergents and soaps (other than personal care
Digital computing equipment and devices
Edible fats and oils
Eggs
Eye care goods
Eye care services
Financial services
Fish
Fresh fruit
Fresh or frozen meat (excluding poultry
Fresh or frozen poultry
Fresh vegetables
Furniture
Gasoline
Household appliances
Household textiles
Housekeeping services
Internet access services
Liquor purchased from stores
Liquor served in licensed establishments
Magazines and periodicals
Medicinal and pharmaceutical products
Newspapers
Non-alcoholic beverages
Non-electric kitchen utensils, tableware and cookware
Nuts and seeds
Operation of recreational vehicles
Oral-hygiene produ

# Loss Analysis 3:

No Loss Term 3:

In [18]:
weightspath_3

'/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Canada/bidirectional/models_weights_3/'

In [19]:
def bi_directional_model_3(son_parent_dict, parent_to_son_list_dict, train_dataset_dict, test_dataset_dict, categories_per_indent_dict, category_id_to_name_dict, weights_path, hgru_weight_dict, coefficient_dict,loss_coef_1, loss_coef_2, loss_coef_3, alpha):
    # deleting files in models_weights folder
    path = os.getcwd()
    desired_path = path + '/models_weights_3/'
    is_empty = not any(Path(desired_path).iterdir())

    if is_empty==False:
        rmdir(Path(desired_path))
    
    # deleting empty folder
    try:
        os.rmdir(desired_path)
    except OSError:
        print ("Deletion of the directory %s failed" % desired_path)
    else:
        print ("Successfully deleted the directory %s" % desired_path)

    # creating new folder
    try:
        os.mkdir(desired_path)
    except OSError:
        print ("Creation of the directory %s failed" % desired_path)
    else:
        print ("Successfully created the directory %s" % desired_path)
    #-----------------------------------------------------------------------------------------------------------------------------------------------#

    num_categories = 0
    loss_coef_1= 1*np.exp(-10)
    loss_coef_2= 1*np.exp(-10)
    loss_coef_3= 0

    for indent in sorted(list(categories_per_indent_dict.keys()),reverse=True):
        for category in categories_per_indent_dict[indent]:
            num_categories +=1
            print(f'num categories: {num_categories}')
            category_name = category_id_to_name_dict[category]
            print(f'category id|name: {category}|{category_name}')

            if int(indent) == 0 or son_parent_dict[category] not in categories_per_indent_dict[indent-1]: #or (category, son_parent_dict[category]) not in son_parent_corr_dict.keys():
                loss_coef_1 = 0

            if (category not in list(parent_to_son_list_dict.keys())) or (set(parent_to_son_list_dict[category]['sons']).issubset(set(categories_per_indent_dict[indent+1]))):
                loss_coef_2 = 0
            
            print('------------------------------------------------------------------')

            train_dataloader, test_dataloader = create_dataloader(train_dataset_dict[category_name], test_dataset_dict[category_name])
            model = GRUModel(input_dim=Features, hidden_dim=HiddenSize, layer_dim=LayersDim, output_dim=OutputDim, dropout_prob=DropoutProb, seed=0)
            optimizer = torch.optim.AdamW(model.parameters(), lr=Lr)
            model.to(Device)
            saving_param_path = weights_path+category_name+'.pt'
            
            training_and_evaluation(model, indent, train_dataloader, test_dataloader, optimizer, category, hgru_weight_dict, coefficient_dict, son_parent_dict, parent_to_son_list_dict, category_id_to_name_dict, loss_coef_1, loss_coef_2, loss_coef_3, alpha, saving_param_path)

In [20]:
bidrectional_models_3 = bi_directional_model_3(son_parent_dict, parent_to_son_list_dict, train_dataset_dict, test_dataset_dict, categories_per_indent_dict, category_id_to_name_dict, weightspath_3, hgru_weight_dict, coefficient_dict, loss_coef_1, loss_coef_2, 0, alpha)

Deletion of the directory /Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Canada/bidirectional/models_weights_3/ failed
Successfully created the directory /Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Canada/bidirectional/models_weights_3/
num categories: 1
category id|name: 345|All other miscellaneous food preparations
------------------------------------------------------------------
num categories: 2
category id|name: 316|Bacon
------------------------------------------------------------------
num categories: 3
category id|name: 332|Canned fruit
------------------------------------------------------------------
num categories: 4
category id|name: 341|Canned infant or junior foods
------------------------------------------------------------------
num categories: 5
category id|name: 318|Canned salmon
------------------------------------------------------------------
num categories: 6
category id|name: 319|Canned tuna
----------------------------------

In [21]:
predictions_dict_3 = get_results_on_test_set(weightspath_3, train_dataset_dict, test_dataset_dict, categories = categories)
with open(test_predictions_path_3, 'wb') as handle:
    pickle.dump(predictions_dict_3, handle, protocol=pickle.HIGHEST_PROTOCOL)

Air transportation
Audio equipment
Bakery products
Beer purchased from stores
Beer served in licensed establishments
Books and reading material (excluding textbooks
Cereal products (excluding baby food
Child care services
City bus and subway transportation
Coffee and tea
Condiments, spices and vinegars
Dairy products
Dental care services
Detergents and soaps (other than personal care
Digital computing equipment and devices
Edible fats and oils
Eggs
Eye care goods
Eye care services
Financial services
Fish
Fresh fruit
Fresh or frozen meat (excluding poultry
Fresh or frozen poultry
Fresh vegetables
Furniture
Gasoline
Household appliances
Household textiles
Housekeeping services
Internet access services
Liquor purchased from stores
Liquor served in licensed establishments
Magazines and periodicals
Medicinal and pharmaceutical products
Newspapers
Non-alcoholic beverages
Non-electric kitchen utensils, tableware and cookware
Nuts and seeds
Operation of recreational vehicles
Oral-hygiene produ

# Loss Analysis 1+2:

In [16]:
weightspath_1_2

'/Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Canada/bidirectional/models_weights_1_2/'

In [10]:
def bi_directional_model_1_2(son_parent_dict, parent_to_son_list_dict, train_dataset_dict, test_dataset_dict, categories_per_indent_dict, category_id_to_name_dict, weights_path, hgru_weight_dict, coefficient_dict,loss_coef_1, loss_coef_2, loss_coef_3, alpha):
    # deleting files in models_weights folder
    path = os.getcwd()
    desired_path = path + '/models_weights_2/'
    is_empty = not any(Path(desired_path).iterdir())

    if is_empty==False:
        rmdir(Path(desired_path))
    
    # deleting empty folder
    try:
        os.rmdir(desired_path)
    except OSError:
        print ("Deletion of the directory %s failed" % desired_path)
    else:
        print ("Successfully deleted the directory %s" % desired_path)

    # creating new folder
    try:
        os.mkdir(desired_path)
    except OSError:
        print ("Creation of the directory %s failed" % desired_path)
    else:
        print ("Successfully created the directory %s" % desired_path)
    #-----------------------------------------------------------------------------------------------------------------------------------------------#

    num_categories = 0
    loss_coef_1= 0
    loss_coef_2= 0
    loss_coef_3= 6.16933911191657*np.exp(-8)
    alpha= 1.0

    for indent in sorted(list(categories_per_indent_dict.keys()),reverse=True):
        for category in categories_per_indent_dict[indent]:
            num_categories +=1
            print(f'num categories: {num_categories}')
            category_name = category_id_to_name_dict[category]
            print(f'category id|name: {category}|{category_name}')

            if int(indent) == 0 or son_parent_dict[category] not in categories_per_indent_dict[indent-1]: #or (category, son_parent_dict[category]) not in son_parent_corr_dict.keys():
                loss_coef_1 = 0

            if (category not in list(parent_to_son_list_dict.keys())) or (set(parent_to_son_list_dict[category]['sons']).issubset(set(categories_per_indent_dict[indent+1]))):
                loss_coef_2 = 0
            
            print('------------------------------------------------------------------')

            train_dataloader, test_dataloader = create_dataloader(train_dataset_dict[category_name], test_dataset_dict[category_name])
            model = GRUModel(input_dim=Features, hidden_dim=HiddenSize, layer_dim=LayersDim, output_dim=OutputDim, dropout_prob=DropoutProb, seed=0)
            optimizer = torch.optim.AdamW(model.parameters(), lr=Lr)
            model.to(Device)
            saving_param_path = weights_path+category_name+'.pt'
            
            training_and_evaluation(model, indent, train_dataloader, test_dataloader, optimizer, category, hgru_weight_dict, coefficient_dict, son_parent_dict, parent_to_son_list_dict, category_id_to_name_dict, loss_coef_1, loss_coef_2, loss_coef_3, alpha, saving_param_path)

In [11]:
bi_directional_model_1_2 = bi_directional_model_1_2(son_parent_dict, parent_to_son_list_dict, train_dataset_dict, test_dataset_dict, categories_per_indent_dict, category_id_to_name_dict, weightspath_1_2, hgru_weight_dict, coefficient_dict, 0, 0, loss_coef_3, alpha)

Deletion of the directory /Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Canada/bidirectional/models_weights_2/ failed
Successfully created the directory /Users/mvilenko/Library/CloudStorage/OneDrive-PayPal/hgru_clean/Canada/bidirectional/models_weights_2/
num categories: 1
category id|name: 345|All other miscellaneous food preparations
------------------------------------------------------------------
num categories: 2
category id|name: 316|Bacon
------------------------------------------------------------------
num categories: 3
category id|name: 332|Canned fruit
------------------------------------------------------------------
num categories: 4
category id|name: 341|Canned infant or junior foods
------------------------------------------------------------------
num categories: 5
category id|name: 318|Canned salmon
------------------------------------------------------------------
num categories: 6
category id|name: 319|Canned tuna
----------------------------------

In [14]:
predictions_dict_1_2 = get_results_on_test_set(weightspath_1_2, train_dataset_dict, test_dataset_dict, categories = categories)
with open(test_predictions_path_1_2, 'wb') as handle:
    pickle.dump(predictions_dict_1_2, handle, protocol=pickle.HIGHEST_PROTOCOL)

Air transportation
Audio equipment
Bakery products
Beer purchased from stores
Beer served in licensed establishments
Books and reading material (excluding textbooks
Cereal products (excluding baby food
Child care services
City bus and subway transportation
Coffee and tea
Condiments, spices and vinegars
Dairy products
Dental care services
Detergents and soaps (other than personal care
Digital computing equipment and devices
Edible fats and oils
Eggs
Eye care goods
Eye care services
Financial services
Fish
Fresh fruit
Fresh or frozen meat (excluding poultry
Fresh or frozen poultry
Fresh vegetables
Furniture
Gasoline
Household appliances
Household textiles
Housekeeping services
Internet access services
Liquor purchased from stores
Liquor served in licensed establishments
Magazines and periodicals
Medicinal and pharmaceutical products
Newspapers
Non-alcoholic beverages
Non-electric kitchen utensils, tableware and cookware
Nuts and seeds
Operation of recreational vehicles
Oral-hygiene produ

In [22]:
# Sanity Tests:

In [24]:
predictions_dict_1['Tomatoes']

tensor([[ -3.8733, -15.6285, -14.5590, -11.3619,  -0.6094,   8.3143,   5.6931,
          -0.1745,   9.2447,  11.2913,  10.8404,   5.8491,   6.3514,  -5.0993,
          -7.7513, -12.1553,  -0.9966,   4.1364,   6.3052,   5.3701,   4.3439,
           6.5750,   4.5572,   8.2177,   9.4795]])

In [24]:
predictions_dict_2['Food']

tensor([[0.6280, 0.5698, 0.7152, 0.7166, 0.6874, 0.5765, 0.4632, 0.2429, 0.6959,
         0.8016, 0.6461, 0.8040, 0.7555, 0.6798, 0.6602, 0.8281, 0.8073, 0.7892,
         0.8522, 0.8486, 0.8944, 0.9425, 0.9445, 0.9647, 0.9326]])

In [25]:
predictions_dict_3['Food']

tensor([[0.7351, 0.4353, 0.5155, 0.6934, 0.5405, 0.5435, 0.5324, 0.3236, 0.4081,
         0.6848, 0.6279, 0.5743, 0.6398, 0.5539, 0.5360, 0.5869, 0.7615, 0.6301,
         0.7049, 0.6692, 0.7178, 0.7942, 0.8015, 0.8563, 0.7888]])

In [25]:
predictions_dict_1_2['Tomatoes']

tensor([[ -3.8733, -15.6285, -14.5590, -11.3619,  -0.6094,   8.3143,   5.6931,
          -0.1745,   9.2447,  11.2913,  10.8404,   5.8491,   6.3514,  -5.0993,
          -7.7513, -12.1553,  -0.9966,   4.1364,   6.3052,   5.3701,   4.3439,
           6.5750,   4.5572,   8.2177,   9.4795]])